In [2]:
import os
import joblib
import pandas as pd


dataset_type = "full" 
model_dir    = f"models/models_{dataset_type}"
model_path   = os.path.join(model_dir, "best_ngb_default_model_MatImputer.pkl")
scaler_path  = os.path.join(model_dir, "scaler_MatImputer.pkl")
test_csv     = "experimental_test.csv"
out_csv      = "experimental_test_with_predictions.csv"


ngb_model = joblib.load(model_path)
scaler    = joblib.load(scaler_path)


df_test = pd.read_csv(test_csv)

features = [
    'Li_fraction_in_TM_layer', 'Ni_fraction', 'Co_fraction', 'Mn_fraction',
    'first_sintering_temperature', 'first_sintering_time',
    'second_sintering_temperature', 'second_sintering_time'
]

X_test        = df_test[features]
X_test_scaled = scaler.transform(X_test)
y_dist        = ngb_model.pred_dist(X_test_scaled)

df_test['predicted_mean'] = y_dist.loc
df_test['predicted_std']  = y_dist.scale
df_test['lower_95']       = y_dist.ppf(0.025)
df_test['upper_95']       = y_dist.ppf(0.975)


print(df_test[['predicted_mean','predicted_std','lower_95','upper_95']].head())
df_test.to_csv(out_csv, index=False)
print(f"\nSaved predictions to {out_csv}")


   predicted_mean  predicted_std  lower_95  upper_95
0        0.206419       0.068101  0.072944  0.339894
1        0.363823       0.074039  0.218708  0.508937
2        0.123007       0.059365  0.006655  0.239360
3        0.170339       0.058784  0.055125  0.285553

Saved predictions to experimental_test_with_predictions.csv
